In [13]:
from queue import PriorityQueue
import math
import pickle
import numpy as np

In [14]:
import pickle
def get_data_from_pickle(x):
    with open(x, 'rb') as f:
        result = pickle.load(f)
    return result

In [15]:
centroid_vectors = get_data_from_pickle("kmeans15.cluster_centers_.pickle")
tag_dict = get_data_from_pickle("tag_dict.pickle")
tag_numbering = get_data_from_pickle("tag_numbering.pickle")
tag_vectors = get_data_from_pickle("tag_vectors")
unique_tag_list = get_data_from_pickle("unique_tag_list.pickle")

In [16]:
from queue import PriorityQueue

def distance_tag_to_centroid(tag_idx,cluster_idx): # tag와 cluster의 중심까지의 거리
    return np.sum((tag_vectors[tag_idx]-centroid_vectors[cluster_idx])**2)

def nearest_clusters(tag_idx):    # tag와 가장 가까운 클러스터 1개 또는 2개 출력하는 함수
    que = PriorityQueue(maxsize=15)
    near_list_idx = []
    near_clusters_dict = dict()
    for i in range(15):    # 가장 거리가 짧은 클러스터 순서대로 정렬
        que.put([distance_tag_to_centroid(tag_idx,i),i])
    for i in range(2):     # 거리가 짧은 2개만 추출
        near_list_idx.append(que.get())
    
    if(2*near_list_idx[0][0]<near_list_idx[1][0]):    # a(첫번째 클러스터와 태그의 거리), b(두번째 클러스터와 태그의 거리)라고 할 때, a*2 < b 이면 두번째 클러스터 제거(즉 거리차이가 많이나면 제거)
        del near_list_idx[1]
        
    if(len(near_list_idx)==1):
        near_clusters_dict[near_list_idx[0][1]] = near_list_idx[0][0]    
    else:
        for i in range(2):
            near_clusters_dict[near_list_idx[i][1]] = near_list_idx[i][0]        
    
    return near_clusters_dict

In [17]:
import math

def clusters_and_weight(tag_idx):     # 태그의 인덱스를 넣으면 원하는 결과가 출력 --> dict{cluster_index : weight}
    ans_dict = dict()
    near_cluster = nearest_clusters(tag_idx)
    dict_keys = list(near_cluster.keys())
    
    if(len(dict_keys)==1):
        ans_dict[dict_keys[0]] = 1.0
    else:
        a = 2/near_cluster[dict_keys[0]]
        b = 1/near_cluster[dict_keys[1]]
        
        a_percentage = round(a/(a+b),2)
        b_percentage = round(1-a_percentage,2)
        
        ans_dict[dict_keys[0]] = a_percentage
        ans_dict[dict_keys[1]] = b_percentage     
    return ans_dict

In [22]:
clusters_and_weight(3)

{7: 1.0}